### CSS selector

- 아무것도 없으면, 태그명
- .(마침표) 는: 클래스명
- #은 :id 
- 분여놓으면 adn 조건
- 공백을 띄우면 하위 태그 
- 태그: nth-child 
  ex. tr:nth ->nth-of-type(2) => tr의 하위 태그 td에서 td 중에서 2번째 자식들만 선택

### 1. Python으로 웹페이지의 HYML 소스 받기

In [10]:
import requests
raw = requests.get("http://147.46.178.16:33333/table.html", headers={'User-Agent': 'Mozilla/5.0'})
print(raw.text)

<table border=“1“>
  <tr>
    <th class="exampleClass1">요일</th>
    <th>PM10</th>
  </tr>
  <tr>
    <td>월</td>
    <td>100</td>
  </tr>
  <tr>
    <td>화</td>
    <td>80</td>
  </tr>
  <tr>
    <td>수</td>
    <td>180</td>
  </tr>
  <tr>
    <td>목</td>
    <td class="exampleClass1">280</td>
  </tr>
  <tr>
    <td id="exampleID1">금</td>
    <td>50</td>
  </tr>
</table>



<style>
.exampleClass1 {
	color: blue;
	background-color: yellow;
}

#exampleID1 {
	color: red;
	background-color: blue;
	font-family: Verdana;
}
</style>


### 2. BeautifulSoup를 사용해서 소스 분석
- raw data를 beautifulsoup를 이용해서, html parsing

In [9]:
from bs4 import BeautifulSoup 
html = BeautifulSoup(raw.content, "html.parser", from_encoding="utf-8") 
print(html)

<table border="“1“">
<tr>
<th class="exampleClass1">요일</th>
<th>PM10</th>
</tr>
<tr>
<td>월</td>
<td>100</td>
</tr>
<tr>
<td>화</td>
<td>80</td>
</tr>
<tr>
<td>수</td>
<td>180</td>
</tr>
<tr>
<td>목</td>
<td class="exampleClass1">280</td>
</tr>
<tr>
<td id="exampleID1">금</td>
<td>50</td>
</tr>
</table>
<style>
.exampleClass1 {
	color: blue;
	background-color: yellow;
}

#exampleID1 {
	color: red;
	background-color: blue;
	font-family: Verdana;
}
</style>


#### 방법 1) CSS select로 추출

In [24]:
print(html.select_one("tr th"))
print(html.select_one("tr th").text)
print(html.select_one("tr th").get('class'))

<th class="exampleClass1">요일</th>
요일
['exampleClass1']


In [57]:
result = html.select("tr th")
print(len(result))
print(result[0])
print(result[1])

for r in result:
    print(r.text)
result2 = html.select("tr td:nth-of-type(2)")

2
<th class="exampleClass1">요일</th>
<th>PM10</th>
요일
PM10


In [51]:
links = html.find_all() 
# links = html.find_all({'class':'exampleClass1'}) 
links = html.select(".exampleClass1") # 클래스가 examleClass2 인 것ㅇ르 불러오기 

#### 방법 2) Beautiful soup 내장 함수 이용

In [79]:
#  packagae download

from selenium import webdriver
from bs4 import BeautifulSoup

# driver path 
driver_path= 'C:/Users/ghkdn/Jupyter/chromedriver_win32/chromedriver' 
wd = webdriver.Chrome(executable_path=driver_path)  # Chrome driver 
wd.get("http://147.46.178.16:33333/table.html")

In [81]:
print(wd.find_element_by_css_selector("tr th").text)
print(wd.find_element_by_css_selector("tr th").get_attribute("class"))

# find_element는 1개, find_elements는 여러개 
rows = wd.find_elements_by_css_selector("tr td")
for i in rows:
    print(i.text)

요일
exampleClass1
월
100
화
80
수
180
목
280
금
50


### java script.html 동적 스크립트 값 가져오기 
- 동적자료는 selector로 안되기 때문에, selenium을 이용해야함

In [84]:
import time
# driver path 
driver_path= 'C:/Users/ghkdn/Jupyter/chromedriver_win32/chromedriver' 
wd = webdriver.Chrome(executable_path=driver_path)  # Chrome driver 
wd.get("http://147.46.178.16:33333/javascript.html")
time.sleep(1)

wd.find_element_by_css_selector("body a").click()
print(wd.page_source)

<html><head></head><body><a href="javascript:openComments();">로또번호 생성하기</a>
<div id="commentArea">18<br>35<br>43<br>44<br>9<br>29<br>4<br></div>

<script>
	var ddd = Math.random()
  function openComments() {
    a = "";
    for (i = 0; i < 7; i++) {
      a += Math.ceil(Math.random() * 45) + "<br>";
    }
    document.getElementById("commentArea").innerHTML = a;
  }

  function displayTime() {
    document.getElementById('demo').innerHTML = Date()
  }

</script></body></html>


## Crawling 활용

1. HTML 소스 가져오기 (selenium or requests) 
2. 추출할 아이템(들)을 정하고, 이를 공통적으로 포함하는 박스 정하기 
3. 박스의 selector 파악하기 
4. 아이템(들)의 selector 파악하기 
5. for문 내에서 각 박스별로 아이템 추출 및 출력/저장
6. 필요시 다음 페이지/키워드로 넘어가기

    - 링크: https://sports.news.naver.com/index.nhn 

In [166]:
#  packagae download

from selenium import webdriver
from bs4 import BeautifulSoup
import time
import pandas as pd

driver_path= 'C:/Users/ghkdn/Jupyter/chromedriver_win32/chromedriver' 
url= 'https://sports.news.naver.com/index.nhn'

wd = webdriver.Chrome(executable_path=driver_path)  # Chrome driver 
wd.get(url)
print(wd.page_source)
# wd.quit()

<html lang="ko"><head>
	<meta charset="UTF-8">
	<meta http-equiv="X-UA-Compatible" content="IE=edge">
	<meta http-equiv="X-UA-Compatible" content="requiresActiveX=true">
	<meta name="viewport" content="width=1080px,maximum-scale=2.0,minimum-scale=0.4,user-scalable=yes">
	<meta property="og:image" content="https://imgsports.pstatic.net/images/mobile/common/NAVERsports.png">
	<meta property="og:title" content="스포츠홈 : 네이버 스포츠">
	<meta property="og:description" content="스포츠의 시작과 끝!">
	<meta name="description" content="스포츠의 모든 것, 네이버 스포츠와 함께 하세요">
	
		
		
		
		
	
	<title>스포츠홈 : 네이버 스포츠</title>
	
    <link rel="stylesheet" type="text/css" href="https://ssl.pstatic.net/static.sports/resources/pc/2020/07/13/161344/css/generated/nsportsCss.css">

    <link rel="stylesheet" type="text/css" href="https://ssl.pstatic.net/static.sports/resources/pc/2020/07/13/161344/css/generated/nsportsHome.css">


	<link rel="shortcut icon" href="https://www.naver.com/favicon.ico?v=20140228" type="image/x-icon">


In [167]:
boxes = wd.find_elements_by_css_selector(".link_today div.text_area")
print(len(boxes))

6


In [168]:
for box in boxes:
    print(box.find_element_by_css_selector("a strong").text)

[단독] SK 퓨처스팀 선수간 폭행·무면허 운전 의혹…논란 일자 뒤늦게 KBO에 보고
'역시 빛현우' 조현우, 올시즌 연속 클린시트 최다 기록 보유
'낮잠을 자?'...레알, 베일에게 '2년치 연봉' 주고 방출한다
[고척 프리뷰] 드디어 돌아온 브리검, 키움 '난세의 영웅'이 필요하다
클롭 감독 "2024년 리버풀 떠나겠다" 폭탄 발언
[현장이슈] 상식 벗어난 해명...논란만 키운 KFA의 '답정너' 브리핑


In [141]:
numSuccess = 0
numFailure = 0
string_dict = dict()

for num,box in enumerate(boxes):
    try:
        item = box.find_element_by_css_selector("a strong").text
        string_dict["theme"+str(num)] = item
        category
    except:
        print("error")
        numFailure +=1
    else:
        print(item)
        numSuccess +=1
print("성공:", numSuccess, "실패:", numFailure)

[단독] SK 퓨처스팀 선수간 폭행·무면허 운전 의혹…논란 일자 뒤늦게 KBO에 보고
'역시 빛현우' 조현우, 올시즌 연속 클린시트 최다 기록 보유
'낮잠을 자?'...레알, 베일에게 '2년치 연봉' 주고 방출한다
[고척 프리뷰] 드디어 돌아온 브리검, 키움 '난세의 영웅'이 필요하다
홀란드, 클럽 갔다가 '퇴짜'…아버지 "집에 와 일이나 해라"
[현장이슈] 상식 벗어난 해명...논란만 키운 KFA의 '답정너' 브리핑
성공: 6 실패: 0


In [142]:
string_dict

{'theme0': '[단독] SK 퓨처스팀 선수간 폭행·무면허 운전 의혹…논란 일자 뒤늦게 KBO에 보고',
 'theme1': "'역시 빛현우' 조현우, 올시즌 연속 클린시트 최다 기록 보유",
 'theme2': "'낮잠을 자?'...레알, 베일에게 '2년치 연봉' 주고 방출한다",
 'theme3': "[고척 프리뷰] 드디어 돌아온 브리검, 키움 '난세의 영웅'이 필요하다",
 'theme4': '홀란드, 클럽 갔다가 \'퇴짜\'…아버지 "집에 와 일이나 해라"',
 'theme5': "[현장이슈] 상식 벗어난 해명...논란만 키운 KFA의 '답정너' 브리핑"}

In [163]:
data = (pd.DataFrame.from_dict([string_dict])).T
data

,0
theme0,[단독] SK 퓨처스팀 선수간 폭행·무면허 운전 의혹…논란 일자 뒤늦게 KBO에 보고
theme1,"'역시 빛현우' 조현우, 올시즌 연속 클린시트 최다 기록 보유"
theme2,"'낮잠을 자?'...레알, 베일에게 '2년치 연봉' 주고 방출한다"
theme3,"[고척 프리뷰] 드디어 돌아온 브리검, 키움 '난세의 영웅'이 필요하다"
theme4,"홀란드, 클럽 갔다가 '퇴짜'…아버지 ""집에 와 일이나 해라"""
theme5,[현장이슈] 상식 벗어난 해명...논란만 키운 KFA의 '답정너' 브리핑


####  강사님 코드 구현 예시

In [174]:
#  packagae download

from selenium import webdriver
from bs4 import BeautifulSoup
import time
import pandas as pd

driver_path= 'C:/Users/ghkdn/Jupyter/chromedriver_win32/chromedriver' 
wd = webdriver.Chrome(executable_path=driver_path)  # Chrome driver 
wd.get(url)


wd.get("https://sports.news.naver.com/index.nhn")
time.sleep(2)

boxes = wd.find_elements_by_css_selector(".today_item")
# print(len(boxes))
for box in boxes:
    category = box.find_element_by_css_selector(".information span:nth-of-type(2)").text 
    # nth-of-type는 인덱스가 0부터가 아닌 1부터 시작
    news = box.find_element_by_css_selector(".information span:nth-of-type(1)").text
    title = box.find_element_by_css_selector("strong.title").text
    print(news + "\t" + category + "\t" + title)

엠스플뉴스	KBO리그	[단독] SK 퓨처스팀 선수간 폭행·무면허 운전 의혹…논란 일자 뒤늦게 KBO에 보고
풋볼리스트	K리그	'역시 빛현우' 조현우, 올시즌 연속 클린시트 최다 기록 보유
스포티비뉴스	프리미어리그	클롭 감독 "2024년 리버풀 떠나겠다" 폭탄 발언
인터풋볼	라리가	'낮잠을 자?'...레알, 베일에게 '2년치 연봉' 주고 방출한다
스포츠조선	KBO리그	[고척 프리뷰] 드디어 돌아온 브리검, 키움 '난세의 영웅'이 필요하다
스포탈코리아	K리그	[현장이슈] 상식 벗어난 해명...논란만 키운 KFA의 '답정너' 브리핑
